# Welcome to the Synergy Single Frame with Image Streamer POC
Created by Fredrik Tärnell - 2019-02-29<br>

Jupyter Notebooks can be found: https://github.com/frippe75/synergy-jupyter-notebooks<br>
Together with a Docker image with all dependencies inluded and a quickstart guide.


### API information

http://h17007.www1.hpe.com/docs/enterprise/servers/oneview4.1/cic-api/en/index.html <br>


### Python specific
https://hewlettpackard.github.io/python-hpOneView/index.html<br>
https://github.com/HewlettPackard/python-hpOneView<br>

## Table of content

- **[Dependencies and logging in](#Dependencies)**<br>
   + [Import the Python OneView library](#Import)<br>
   + [YAML file or embedded document](#YAML)<br>
   + [Login against Oneview](#Login)<br>
   
- **[Settings](#Setting)**<br>
   + [Addresses and Indentifiers](#Address-identifiers)<br>
   + [Networks](#Networks)<br>

- **[Storage](#Storage)**<br>
   + [SAN Managers](#SANMgr)<br> 
   + [Storage Systems](#StorageSystems)<br> 
   


  <br> 
- **[Server Profiles](#SP)**<br>
   + [Power On/Off all Profiles](#SP-power)<br>
   + [Delete all Profiles](#SP-delete)<br>
- **[Server Profiles Templates](#SPT)**<br>
   + [Delete all Profiles Templates](#SPT-delete)<br>
- **[Appliances](#Appliances)**<br>
   + [eFuse Composer / Image Streamer](#Appliance-efuse)<br>

<br><br><br>
# Dependencies and logging in <a name="Dependencies"></a>



### Import the Python OneView library with some additional dependencies for this playbook <a name="Import"></a>

In [8]:
# Mandatory ones
from hpOneView.oneview_client import OneViewClient
from hpOneView.exceptions import HPOneViewException

# Nice to haves for the demo
import pandas, yaml
from IPython.display import display, Markdown
from pprint import pprint

poc_file = None

# Simple helper function to make nicer tables using markdown and the pandas lib
# TODO: get it into helper module that gets imported above 
def my_table(source=dict(), resource="Resource", columns='name'):
    data = pandas.DataFrame.from_dict(source)
    table = data.to_html(index=False,header=True,columns=columns)
    
    display(Markdown('### ' + resource + ' in Synergy Composer at https://' + config['ip']))
    display(Markdown(table))

### Configure your environment
**Please note:** that if you use the option below to get the YAML doc from a file the config dict will be overwritten by the config from the file. During the login, check that you are logged in against the correct Oneview instance

In [2]:
composer_ip = "synergy.hpedemo.local"
config = {
    "api_version": "800",
    "ip": composer_ip,
    "credentials": {
        "userName": "Administrator",
        "authLoginDomain": "local",
        "password": "*******"
    }
}

### YAML file or embedded document <a name="YAML"></a>
Determine wheter to load the YAML doc from a variable or file<br>
Setting the filename takes precedence over variable

In [24]:
# Comment out to use the poc_file to allow the embedded poc_doc to be used
poc_file = "synergy-masters.yaml"

# Example YAML stucture for allowing Addresses and Indentifiers part work without a file
poc_doc = """
---
  addresses:
    # Synergy Management 
    - networkId: 10.48.0.0
      subnetmask: 255.255.255.0
      gateway: 10.48.0.1
      domain: hpedemo.local
      dnsServers: 
        - 10.48.10.4
        - 10.48.10.6
      range:
        name: LE-POC-01
        startAddress: 10.48.0.104
        endAddress: 10.48.0.124
    # Image Streamer iSCSI 
    - networkId: 192.168.10.0
      subnetmask: 255.255.255.0
      range:
        name: iSCSI-boot
        startAddress: 192.168.10.10
        endAddress: 192.168.10.100
    # VMware Mgmt 
    - networkId: 10.48.10.0
      subnetmask: 255.255.255.0
      gateway: 10.48.10.1
      domain: hpedemo.local
      dnsServers: 
        - 10.48.10.4
        - 10.48.10.6
      range:
        name: LE-POC-01
        startAddress: 10.48.10.70
        endAddress: 10.48.10.79
    # vMotion 
    - networkId: 192.168.20.0
      subnetmask: 255.255.255.0
      range:
        name: vMotion
        startAddress: 192.168.20.10
        endAddress: 192.168.20.100
"""
# load/parse the YAML doc into a variable
if poc_file is not None:
    with open(poc_file) as f:
        poc_doc = f.read()
    # print(poc_doc)
    poc = yaml.load(poc_doc)
    # Check if Oneview config key is part of the YAML file and use that for logging in.
    if 'config' in poc:
        print("\nOneview endpoint config was found and is used. Removing it to avoid displaying passwords\n")
        config = poc.pop('config', None)
else:
    poc = yaml.load(poc_doc)
    
# Print it out for sanitycheck
print (yaml.dump(poc))


Oneview endpoint config was found and is used. Removing it to avoid displaying passwords

SANManagers:
- authProtocol: MD5
  ip: 172.18.20.1
  password: dcsdcsdcs
  securityLevel: Authentication
  type: Cisco
  userName: dcs-MD5
addresses:
- dnsServers:
  - 10.10.20.1
  domain: synergy.local
  gateway: 10.10.20.1
  networkId: 10.10.20.0
  range:
    endAddress: 10.10.21.199
    name: LE-G2
    startAddress: 10.10.21.133
  subnetmask: 255.255.252.0
- networkId: 192.168.10.0
  range:
    endAddress: 192.168.10.100
    name: iSCSI-boot
    startAddress: 192.168.10.10
  subnetmask: 255.255.255.0
enclosureGroups:
- ligs:
  - name: My-VC-01
  - name: My-SAS-01
  name: My-EG-01
logicalInterconnectGroups:
- name: My-VC-01
- name: My-SAS-01
networks:
- association: 10.10.20.0
  name: Mgmt_100
  purpose: Management
  vlanId: 100
- association: 192.168.10.0
  name: ImageStreamer-Boot
  purpose: ISCSI
  vlanId: 3001
osDeploymentServers:
- description: My First deploy
  mgmtNetwork: Mgmt_100
  nam

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:52: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.


<br><br>
## Login against the Appliance / Composer instance: <a name="Login"></a>
Create new object "ov" with the config (see above) using the JSON config dict above

In [10]:
try:
    ov = OneViewClient(config)
    print("\nLogged in against " + config['ip'] + " successfully\n")
except HPOneViewException as e:
    print("\nLogged in failed:")
    print(e.msg)


Logged in against 10.10.21.110 successfully



<br><br>
# Settings  <a name="Settings"></a>
## Addresses and Indentifiers  <a name="Address-identifiers"></a>
Here we define required networks for both Synergy Management network as well as Image Streamer iSCSi boot network and VMware vMotion.

In [5]:
# Need to reload/parse the YAML doc again since the options.pop mutates the existing dictionary so the contents of the dictionary changes 
poc = yaml.load(poc_doc)

# Loop over the addresses array and create subnets and ranges
if 'addresses' in poc:
    for options in poc['addresses']:

        options.update({
            "type": "Subnet",
            "name": options['networkId']
        })
        # Remove/extract  the range key since it does not belong there.
        range_option = options.pop('range', None)
        #pprint(options)  

        try:
            print("Create IPv4 subnet " + options['networkId'] +" for id pools")
            ipv4_subnet = ov.id_pools_ipv4_subnets.create(options)

            #pprint(range_option)

            range_option.update({
                "type": "Range",
                "rangeCategory": "Custom",
                "subnetUri": ipv4_subnet['uri']
            })
            print("Creating an IPv4 Range for id pools\n")
            ipv4_range = ov.id_pools_ipv4_ranges.create(range_option)
            # pprint(ipv4_range)
        except HPOneViewException as e:
            print("Warning: " + e.msg + "\n")
else:
    print("No Addresses in the poc dict. Nothing created!")    
    
print("\nGetting all IPv4 subnets...")
all_subnets = ov.id_pools_ipv4_subnets.get_all()

#pprint(all_subnets)

my_table(all_subnets, resource="Subnets", columns=['name','networkId','subnetmask','domain','gateway'])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  


Create IPv4 subnet 10.10.20.0 for id pools

Create IPv4 subnet 192.168.10.0 for id pools
Creating an IPv4 Range for id pools


Getting all IPv4 subnets...


### Subnets in Synergy Composer at https://10.10.21.110

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th>name</th>
      <th>networkId</th>
      <th>subnetmask</th>
      <th>domain</th>
      <th>gateway</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>10.10.20.0</td>
      <td>10.10.20.0</td>
      <td>255.255.252.0</td>
      <td>synergy.local</td>
      <td>10.10.20.1</td>
    </tr>
    <tr>
      <td>192.168.10.0</td>
      <td>192.168.10.0</td>
      <td>255.255.255.0</td>
      <td>None</td>
      <td>None</td>
    </tr>
  </tbody>
</table>

<br><br>
## Networks  <a name="Networks"></a>

In [6]:
# Need to reload/parse the YAML doc again since the options.pop mutates the existing dictionary so the contents of the dictionary changes 
poc = yaml.load(poc_doc)

# My defaults
defaults = {
    "ethernetNetworkType": "Tagged",
    "purpose": "General",
    "smartLink": True,
    "privateNetwork": False,
    "type" : "ethernet-networkV4",
    "connectionTemplateUri": None,
    #"bandwidth": {
    #    "maximumBandwidth": 10000,
    #    "typicalBandwidth": 2000
    #}
}

# Loop over the addresses array and create subnets and ranges
if 'networks' in poc:
    for options in poc['networks']:

        # Remove/extract the association key since it does not belong there.
        association_subnet_id = options.pop('association', None)
        if association_subnet_id is not None:
            # TODO: This uses the get_all which returns an array. Best option is to use get(ID) but that is not implemented 
            #       for resource subnet properly. Doc indicates it should.
            subnet = ov.id_pools_ipv4_subnets.get_all(filter='networkId="' + association_subnet_id + '"')
            subnet_uri = subnet[0]['uri']
            # print(subnet_uri)
            options.update({"subnetUri": subnet_uri})

        # Merge with defaults new style of dictonary unpacking since Python 3.5 https://treyhunner.com/2016/02/how-to-merge-dictionaries-in-python/
        net_options = {**defaults, **options}
        # pprint("\n" + net_options)  

        try:
            print("\nCreating ethernet network " + net_options['name'] )
            ethernet_network = ov.ethernet_networks.create(net_options)
            print("Created succesfully.")
        except HPOneViewException as e:
            print("Error during creation of: " + net_options['name'])
            print(e.msg)      
else:
    print("No Networks in the poc dict. Nothing created!")
        
print("\nGetting all IPv4 networks...")
all_nets = ov.ethernet_networks.get_all()

#pprint(all_nets)        
my_table(all_nets, resource="Networks", columns=['name','description','vlanId','uri','purpose'])    

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  



Creating ethernet network ImageStreamer-Boot
Created succesfully.

Getting all IPv4 networks...


### Networks in Synergy Composer at https://10.10.21.110

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th>name</th>
      <th>description</th>
      <th>vlanId</th>
      <th>uri</th>
      <th>purpose</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>Net_13</td>
      <td>None</td>
      <td>13</td>
      <td>/rest/ethernet-networks/04587364-71bc-4123-825...</td>
      <td>General</td>
    </tr>
    <tr>
      <td>Net_20</td>
      <td>None</td>
      <td>20</td>
      <td>/rest/ethernet-networks/0af1a292-3e3a-4153-966...</td>
      <td>General</td>
    </tr>
    <tr>
      <td>Net_14</td>
      <td>None</td>
      <td>14</td>
      <td>/rest/ethernet-networks/1ba54684-2385-4602-a8a...</td>
      <td>General</td>
    </tr>
    <tr>
      <td>Net_19</td>
      <td>None</td>
      <td>19</td>
      <td>/rest/ethernet-networks/374ec04d-822e-4fbb-9cb...</td>
      <td>General</td>
    </tr>
    <tr>
      <td>Net_12</td>
      <td>None</td>
      <td>12</td>
      <td>/rest/ethernet-networks/5d06575a-43d6-4a94-888...</td>
      <td>General</td>
    </tr>
    <tr>
      <td>Net_1020</td>
      <td>None</td>
      <td>1020</td>
      <td>/rest/ethernet-networks/6cb4b496-a50e-4c74-9ad...</td>
      <td>General</td>
    </tr>
    <tr>
      <td>Net_18</td>
      <td>None</td>
      <td>18</td>
      <td>/rest/ethernet-networks/98803094-1ddf-4b39-98e...</td>
      <td>General</td>
    </tr>
    <tr>
      <td>Net_16</td>
      <td>None</td>
      <td>16</td>
      <td>/rest/ethernet-networks/9a696465-363a-4c36-b16...</td>
      <td>General</td>
    </tr>
    <tr>
      <td>ImageStreamer-Boot</td>
      <td>None</td>
      <td>3001</td>
      <td>/rest/ethernet-networks/9b261663-a359-4291-95e...</td>
      <td>ISCSI</td>
    </tr>
    <tr>
      <td>Net_15</td>
      <td>None</td>
      <td>15</td>
      <td>/rest/ethernet-networks/a247beb9-218f-43c9-870...</td>
      <td>General</td>
    </tr>
    <tr>
      <td>Net_17</td>
      <td>None</td>
      <td>17</td>
      <td>/rest/ethernet-networks/aa27fcef-1fc3-4d9c-aa1...</td>
      <td>General</td>
    </tr>
    <tr>
      <td>Net_11</td>
      <td>None</td>
      <td>11</td>
      <td>/rest/ethernet-networks/c5e06b30-38ce-4610-aba...</td>
      <td>General</td>
    </tr>
    <tr>
      <td>Mgmt_100</td>
      <td>None</td>
      <td>100</td>
      <td>/rest/ethernet-networks/d3fd090a-91f2-4a50-af3...</td>
      <td>Management</td>
    </tr>
  </tbody>
</table>

<br><br>
## SAN Managers  <a name="SANMgr"></a>
This part is pretty much a copy from the pythonOneview examples<br>
https://github.com/HewlettPackard/python-hpOneView/blob/master/examples/san_managers.py<br>
<br>
Difference being it uses the YAML structure to created the listed resources as well as takes care of specifying defaults when needed.


In [29]:
# Need to reload/parse the YAML doc again since the options.pop mutates the existing dictionary so the contents of the dictionary changes 
poc = yaml.load(poc_doc)

# Loop over the addresses array and create subnets and ranges
if 'networks' in poc:
    for mgrs in poc['SANManagers']:

        # Handle Cisco in one way
        if 'type' in mgrs and mgrs['type'] == 'Cisco':
            # print(mgrs['type'])

            # passed values or defaults not optional parameters
            if 'snmpPort' in mgrs:
                snmp_port = mgrs['snmpPort']
            else:
                snmp_port = 161

            if 'authProtocol' in mgrs:
                auth_proto = mgrs['authProtocol']
            else:
                auth_proto = "sha"

            if 'privProtocol' in mgrs:
                priv_proto = mgrs['privProtocol']
            else:
                priv_proto = "aes-128"

            options = {
                'providerDisplayName': 'Cisco',
                'connectionInfo': [
                    {
                        'name': 'Host',
                        'displayName': 'Host',
                        'required': True,
                        'value': mgrs['ip'],
                        'valueFormat': 'IPAddressOrHostname',
                        'valueType': 'String'
                    },{
                        'name': 'SnmpPort',
                        'displayName': 'SnmpPort',
                        'required': True,
                        'value': snmp_port,
                        'valueFormat': 'None',
                        'valueType': 'Integer'
                    },{
                        'name': 'SnmpUserName',
                        'displayName': 'SnmpUserName',
                        'required': True,
                        'value': mgrs['userName'],
                        'valueFormat': 'None',
                        'valueType': 'String'
                    },{
                        'name': 'SnmpAuthLevel',
                        'displayName': 'SnmpAuthLevel',
                        'required': True,
                        'value': 'authnopriv',
                        'valueFormat': 'None',
                        'valueType': 'String'
                    },{
                        'name': 'SnmpAuthProtocol',
                        'displayName': 'SnmpAuthProtocol',
                        'required': False,
                        'value': auth_proto,
                        'valueFormat': 'None',
                        'valueType': 'String'
                    },{
                        'name': 'SnmpAuthString',
                        'displayName': 'SnmpAuthString',
                        'required': False,
                        'value': mgrs['password'],
                        'valueFormat': 'SecuritySensitive',
                        'valueType': 'String'
                    },{
                        'name': 'SnmpPrivProtocol',
                        'displayName': 'SnmpPrivProtocol',
                        'required': False,
                        'value': priv_proto,
                        'valueFormat': 'None',
                        'valueType': 'String'
                    },{
                        'name': 'SnmpPrivString',
                        'displayName': 'SnmpPrivString',
                        'required': False,
                        'value': '',
                        'valueFormat': 'SecuritySensitive',
                        'valueType': 'String'
                    }
                ]
            }

        # Handle Brocade in another way
        if 'type' in mgrs and mgrs['type'] == 'Brocade':   
            # passed values or defaults not optional parameters
            if 'port' in mgrs:
                bna_port = mgrs['port']
            else:
                bna_port = 5989

            if 'UseSsl' in mgrs:
                use_ssl = mgrs['UseSsl']
            else:
                use_ssl = True

            options = {
                'providerDisplayName': 'Brocade',
                'connectionInfo': [{
                    'name': 'Host',
                    'value': mgrs['ip']
                },{
                    'name': 'Port',
                    'value': bna_port
                },{
                    'name': 'Username',
                    'value': mgrs['userName']
                },{
                    'name': 'Password',
                    'value': mgrs['password']
                },{
                    'name': 'UseSsl',
                    'value': use_ssl
                }]
            }

        # Handle HPE San Managers not implemented
        if 'type' in mgrs and mgrs['type'] == 'HPE':
            # TODO: Implement this 
            print("WARNING: HPE San Manager types not implemented.")

        try:
            # print(options)
            provider_uri = ov.san_managers.get_provider_uri(mgrs['type'])
            # print(provider_uri)
            print("Adding SAN Manager " + mgrs['ip'] + "of type " + mgrs['type'])
            san_manager = ov.san_managers.add(options, provider_uri)
            print("Created succesfully.")
        except HPOneViewException as e:
            print("Error during creation of SAN Manager: " + mgrs['ip'])
            print(e.msg) 
else:
    print("No SAN Managers in the poc dict. Nothing created!")
    
print("\nGetting all SAN Managers...")
all_san_mgrs = ov.san_managers.get_all()
#pprint(all_san_mgrs)  

# This will fail for Brocade SAN Managers, TODO: do get_all(filter=Brocade) and handle each type separately
if all_san_mgrs:
    my_table(all_san_mgrs, resource="SAN Managers", columns=['deviceManagerUrl','deviceManagerVersion','description','providerDisplayName','state','status']) 
else:
    print("No SAN Managers found")

Adding SAN Manager 172.18.20.1of type Cisco
Error during creation of SAN Manager: 172.18.20.1
Duplicate SAN manager '172.18.20.1' at '/rest/fc-sans/device-managers/16eb8a22-c055-49d9-996e-aa05009c8518' is already imported.
Adding SAN Manager 172.18.20.2of type Cisco
Error during creation of SAN Manager: 172.18.20.2
Duplicate SAN manager '172.18.20.2' at '/rest/fc-sans/device-managers/43aed419-37d4-43d3-8b67-aa05009c88b5' is already imported.

Getting all SAN Managers...


### SAN Managers in Synergy Composer at https://syn420-dcs.hpedemo.local

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th>deviceManagerUrl</th>
      <th>deviceManagerVersion</th>
      <th>description</th>
      <th>providerDisplayName</th>
      <th>state</th>
      <th>status</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>172.18.20.1</td>
      <td>6.2(5)</td>
      <td>MDS 9250i 40 FC 2 IPS 8 FCoE (1 Slot) Chassis</td>
      <td>Cisco</td>
      <td>Managed</td>
      <td>OK</td>
    </tr>
    <tr>
      <td>172.18.20.2</td>
      <td>6.2(5)</td>
      <td>MDS 9250i 40 FC 2 IPS 8 FCoE (1 Slot) Chassis</td>
      <td>Cisco</td>
      <td>Managed</td>
      <td>OK</td>
    </tr>
  </tbody>
</table>

<br><br>
## Storage Systems <a name="StorageSystems"></a>
This will only add the "no Domain" pools and not all of them

In [12]:
# Need to reload/parse the YAML doc again since the options.pop mutates the existing dictionary so the contents of the dictionary changes 
poc = yaml.load(poc_doc)

defaults = {
    "family": 'StoreServ'
}

# Loop over the storage systems array and add them to oneview
if 'storageSystems' in poc:
    for array in poc['storageSystems']:
        pprint (array)
        
        # Merge with defaults new style of dictonary unpacking since Python 3.5 https://treyhunner.com/2016/02/how-to-merge-dictionaries-in-python/
        array_options = {**defaults, **array}
        # pprint("\n" + array_options) 
        
        # Add and update storage system for management
        try:
            print("\nAdding " + array['hostname'] + " to Oneview")
            storage_system = ov.storage_systems.add(array_options)
            print("Succesfully added it.")
        except HPOneViewException as e:
            storage_system = ov.storage_systems.get_by_hostname(array['hostname'])
            if storage_system:
                print("Storage system '%s' was already added.\n   uri = '%s'" %
                      (storage_system['name'], storage_system['uri']))
            else:
                print(e.msg)
        
        # Adds managed domains and managed pools to StoreServ storage systems
        # This is a one-time only action, after this you cannot change the managed values
        if not storage_system['deviceSpecificAttributes']['managedDomain']:
            storage_system['deviceSpecificAttributes']['managedDomain'] = storage_system[
                'deviceSpecificAttributes']['discoveredDomains'][0]
            for pool in storage_system['deviceSpecificAttributes']['discoveredPools']:
                if pool['domain'] == storage_system['deviceSpecificAttributes']['managedDomain']:
                    pool_to_manage = pool
                    storage_system['deviceSpecificAttributes']['discoveredPools'].remove(pool)
                    pprint(pool_to_manage)
                    break
            storage_system['deviceSpecificAttributes']['managedPools'] = [pool_to_manage]
            ov.storage_systems.update(storage_system)
            print("\nUpdated 'managedDomain' to '{}' so storage system can be managed".format(
                  storage_system['deviceSpecificAttributes']['managedDomain']))
            print("\nManaged storage pool '{}' at uri: '{}'".format(storage_system['deviceSpecificAttributes'][
                  'managedPools'][0]['name'], storage_system['deviceSpecificAttributes']['managedPools'][0]['uuid']))

print("\nGetting all storage systems...")
all_arrays = ov.storage_systems.get_all()
#pprint(all_arrays)        

my_table(all_arrays, resource="Storage Systems", columns=['displayName','family', 'hostname','supportsFC','supportsIscsi','state'])                    

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  


{'family': 'StoreServ',
 'hostname': '10.10.20.20',
 'password': '3pardata',
 'username': '3paradmn'}

Adding 10.10.20.20 to Oneview
Storage system 'Synergy3PAR' was already added.
   uri = '/rest/storage-systems/CZ3905NGNP'

Getting all storage systems...


### Storage Systems in Synergy Composer at https://10.10.21.110

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th>displayName</th>
      <th>family</th>
      <th>hostname</th>
      <th>supportsFC</th>
      <th>supportsIscsi</th>
      <th>state</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>Synergy3PAR</td>
      <td>StoreServ</td>
      <td>10.10.20.20</td>
      <td>True</td>
      <td>False</td>
      <td>Managed</td>
    </tr>
  </tbody>
</table>

<br><br>
# Storage operations
## Show configured / existing storage (systems and pools)

In [38]:
storage_systems = ov.storage_systems.get_all()
storage_pools = ov.storage_pools.get_all()

# storage_pool_name = 'FC_r1'
# storage_pools = oneview_client.storage_pools.get_by('name', storage_pool_name)[0]

print("Storage Systems:")
if storage_systems:
    for stor in storage_systems: 
        print(stor['displayName'] + "    URI: " + stor['uri'])
else:
    print("None defined!")
    
print("\nStorage Pools:")
if storage_pools:
    for storpool in storage_pools:
        print(storpool['name'] + "    URI: " + stor['uri'])
else:
    print("None defined!")

Storage Systems:
None defined!

Storage Pools:
None defined!


In [57]:
if storpool is not None: 
    print (storpool)

NameError: name 'storpool' is not defined

In [15]:
ov.logical_interconnect_groups.get_all()

[{'type': 'logical-interconnect-groupV6',
  'uri': '/rest/logical-interconnect-groups/e217624d-bbf1-4e92-b923-812ee84cda6e',
  'category': 'logical-interconnect-groups',
  'eTag': '758fd64b-3a4c-4325-832f-3915e98f0bcb',
  'created': '2019-03-28T12:48:33.264Z',
  'modified': '2019-03-28T12:49:36.857Z',
  'scopesUri': '/rest/scopes/resources/rest/logical-interconnect-groups/e217624d-bbf1-4e92-b923-812ee84cda6e',
  'uplinkSets': [{'networkType': 'Ethernet',
    'networkUris': ['/rest/ethernet-networks/374ec04d-822e-4fbb-9cb0-d1bfe73660fb',
     '/rest/ethernet-networks/d3fd090a-91f2-4a50-af3d-2698e4a34348',
     '/rest/ethernet-networks/9a696465-363a-4c36-b169-d58b8e7d2b00',
     '/rest/ethernet-networks/1ba54684-2385-4602-a8a7-64d77d03aa2f',
     '/rest/ethernet-networks/0af1a292-3e3a-4153-9663-065f70e7fa51',
     '/rest/ethernet-networks/c5e06b30-38ce-4610-aba5-e87ee9be90bc',
     '/rest/ethernet-networks/aa27fcef-1fc3-4d9c-aa16-89eaaa9c59c2',
     '/rest/ethernet-networks/5d06575a-43d6

In [25]:
# Need to reload/parse the YAML doc again since the options.pop mutates the existing dictionary so the contents of the dictionary changes 
poc = yaml.load(poc_doc)

defaults = {
    "category": None,
    "created": None,
    "description": None,
    "eTag": None,
    "uplinkSets": [],
    "modified": None,
    "name": "OneView Test Logical Interconnect Group",
    "state": "Active",
    "status": None,
    "type": "logical-interconnect-groupV6",
    "enclosureType": "SY12000",
    "interconnectMapTemplate": {
        "interconnectMapEntryTemplates": [
            {
                "logicalDownlinkUri": None,
                "logicalLocation": {
                    "locationEntries": [
                        {
                            "relativeValue": "1",
                            "type": "Bay"
                        },
                        {
                            "relativeValue": 1,
                            "type": "Enclosure"
                        }
                    ]
                },
                "permittedInterconnectTypeUri": None
            },
            {
                "logicalDownlinkUri": None,
                "logicalLocation": {
                    "locationEntries": [
                        {
                            "relativeValue": "2",
                            "type": "Bay"
                        },
                        {
                            "relativeValue": 1,
                            "type": "Enclosure"
                        }
                    ]
                },
                "permittedInterconnectTypeUri": None
            },
            {
                "logicalDownlinkUri": None,
                "logicalLocation": {
                    "locationEntries": [
                        {
                            "relativeValue": 3,
                            "type": "Bay"
                        },
                        {
                            "relativeValue": 1,
                            "type": "Enclosure"
                        }
                    ]
                },
                "permittedInterconnectTypeUri": None
            },
            {
                "logicalDownlinkUri": None,
                "logicalLocation": {
                    "locationEntries": [
                        {
                            "relativeValue": 4,
                            "type": "Bay"
                        },
                        {
                            "relativeValue": 1,
                            "type": "Enclosure"
                        }]
                },
                "permittedInterconnectTypeUri": None
            },
            {
                "logicalDownlinkUri": None,
                "logicalLocation": {
                    "locationEntries": [
                        {
                            "relativeValue": 5,
                            "type": "Bay"
                        },
                        {
                            "relativeValue": 1,
                            "type": "Enclosure"
                        }
                    ]
                },
                "permittedInterconnectTypeUri": None
            },
            {
                "logicalDownlinkUri": None,
                "logicalLocation": {
                    "locationEntries": [
                        {
                            "relativeValue": 6,
                            "type": "Bay"
                        },
                        {
                            "relativeValue": 1,
                            "type": "Enclosure"
                        }
                    ]
                },
                "permittedInterconnectTypeUri": None
            },
            {
                "logicalDownlinkUri": None,
                "logicalLocation": {
                    "locationEntries": [
                        {
                            "relativeValue": 7,
                            "type": "Bay"
                        },
                        {
                            "relativeValue": 1,
                            "type": "Enclosure"
                        }
                    ]
                },
                "permittedInterconnectTypeUri": None
            },
            {
                "logicalDownlinkUri": None,
                "logicalLocation": {
                    "locationEntries": [
                        {
                            "relativeValue": 8,
                            "type": "Bay"
                        },
                        {
                            "relativeValue": 1,
                            "type": "Enclosure"
                        }
                    ]
                },
                "permittedInterconnectTypeUri": None
            }
        ]}
}

# Loop over the storage systems array and add them to oneview
if 'logicalInterconnectGroups' in poc:
    for lig in poc['logicalInterconnectGroups']:
        #pprint (lig)
        
        # Merge with defaults new style of dictonary unpacking since Python 3.5 https://treyhunner.com/2016/02/how-to-merge-dictionaries-in-python/
        lig_options = {**defaults, **lig}
        pprint(lig_options) 
        
        # Add and update 
        try:
            print("\nAdding LIG " + lig['name'] + " to Oneview")
            osd_added = ov.logical_interconnect_groups.create(lig_options)
            print("Succesfully added it.")
        except HPOneViewException as e:
            storage_system = ov.os_deployment_servers.get_by('name',osd['name'])
            if storage_system:
                print("OS deployment server '%s' was already added.\n   uri = '%s'" %
                      (storage_system['name'], storage_system['uri']))
            else:
                print(e.msg)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  


{'category': None,
 'created': None,
 'description': None,
 'eTag': None,
 'enclosureType': 'SY12000',
 'interconnectMapTemplate': {'interconnectMapEntryTemplates': [{'logicalDownlinkUri': None,
                                                                'logicalLocation': {'locationEntries': [{'relativeValue': '1',
                                                                                                         'type': 'Bay'},
                                                                                                        {'relativeValue': 1,
                                                                                                         'type': 'Enclosure'}]},
                                                                'permittedInterconnectTypeUri': None},
                                                               {'logicalDownlinkUri': None,
                                                                'logicalLocation': {'locationEntries': [{'re

<br><br>
# OS Deployment Servers
## Add Server

In [12]:
# Need to reload/parse the YAML doc again since the options.pop mutates the existing dictionary so the contents of the dictionary changes 
poc = yaml.load(poc_doc)

defaults = {
    "description": "OS Deployment Server"
}

# Loop over the storage systems array and add them to oneview
if 'osDeploymentServers' in poc:
    for osd in poc['osDeploymentServers']:
        pprint (osd)
        
        # Merge with defaults new style of dictonary unpacking since Python 3.5 https://treyhunner.com/2016/02/how-to-merge-dictionaries-in-python/
        osd_options = {**defaults, **osd}
        # pprint("\n" + array_options) 
        
        # Add and update 
        try:
            print("\nAdding deploymentserver " + osd['name'] + " to Oneview")
            osd_added = ov.os_deployment_servers.add(osd_options)
            print("Succesfully added it.")
        except HPOneViewException as e:
            storage_system = ov.os_deployment_servers.get_by('name',osd['name'])
            if storage_system:
                print("OS deployment server '%s' was already added.\n   uri = '%s'" %
                      (storage_system['name'], storage_system['uri']))
            else:
                print(e.msg)


{'description': 'My First deploy',
 'mgmtNetwork': 'Mgmt_100',
 'name': 'OS-Deploy01'}

Adding deploymentserver OS-Deploy01 to Oneview
Unrecognized JSON field.


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  
